# Running simple Lotka Volterra predator-prey model

In [ ]:
#--- Load necessary modules
#using Pkg
#Pkg.add("Plots")
using DifferentialEquations, Plots             #--- DifferentialEquations: https://docs.sciml.ai/DiffEqDocs/stable/
                                               #--- Plots: https://docs.juliaplots.org/latest/tutorial/

#--- Function of derivatives
function ode_LV!(dN, N, p, t)
    r0, a, e, m = p                            #--- unpack parameters
    dN[1] = N[1] * (r0 - a * N[2])             #--- equation prey
    dN[2] = N[2] * (e * a *  N[1] - m )        #--- equation predator
end


#--- Define the system 
p = (r0 = 0.5, a=0.6, e=0.2, m = 0.1)           #--- the parameters
N_init = [2.,1.]                                #--- the initial values of state variables
tspan = (0.,100.)                               #--- min and max values of simulation time

problem = ODEProblem(ode_LV!, N_init, tspan, p) #--- define the problem

#--- Solve the problem numerical integration
sol = solve(problem) 

#sol = solve(problem,KenCarp4(),saveat = 1:tspan[2]) 

#--- Plot the dynamics
plot(sol, label=["prey" "predator"])

NB: the solution is a 2-element object with densities presented as a vector of abundances  at each time point.

You might wnat to transform it into a matrix.

In [ ]:
sol
sol.t                     #--- the time vector
sol.u;                    #--- the variable dynamics
convert(Array, sol)'      #--- Transform into a matrix. Equivalent to reduce(hcat, sol.u)'


# Generalized Lotka-Volterra model of n-species

This time, we use a matrix for interaction coefficients, AA, and \".\" for operations to each elements of vectors du, u, and r0

In [ ]:
#--- Function of derivatives
function ode_LVG!(du, u, p, t)
    r0, AA = p                     #--- local copy of parameters
    du .= u .* (r0 .+ (AA * u))                                                          #--- equation
end

In [ ]:
# Use of new modules:  Random for seed! function, Distributions for sample function, LinearAlgebra for diagind function 
using Random, Distributions, LinearAlgebra  

#--- Initialize random number generators
#---------------------------------------
Random.seed!(1)  

#--- Define the system 
#---------------------
S = 100                          #--- number of species                  
r0s = randn(S)                   #--- intrinsic growth rates
AAs = -rand(S,S) ./ 100.         #--- interaction coeffficients
AAs[diagind(AAs)] .= - 0.1       #--- intraspecific interaction coefficients

p = (r0s, AAs)                                       #--- the parameters
N_init = sample(range(1., 2.,length = 1000),S)       #--- the initial values of state variables
tspan = (0.,100.)                                    #--- min and max values of simulation time
                                                     # duration = @elapsed begin
prob = ODEProblem(ode_LVG!, N_init, tspan, p)        #--- define the problem


#--- Run numerical integration
#-----------------------------
sol = solve(prob,isoutofdomain = (u, p, t) -> any(x -> x < 0, u));   
                                                     # end
                                                     # @info string("duration = ",duration)
#--- Plot the dynamics
#---------------------
plot(sol, legend=false)


# Using Callbacks

You may want to have events hapening during the dynamics. Julia allows an access to the integration interface during the dynamics

In [ ]:
using DiffEqCallbacks             #--- https://docs.sciml.ai/DiffEqCallbacks/stable/


#--- Callback functions for the perturbations
#--------------------------------------------

#--- Condition for the perturbation
function condition_perturb(u, t, integrator) 
    in(t,(integrator.p[3]))                                            #--- time reaches one of the predefined time points
end

#--- Perturbation function 
function affect_perturb!(integrator) 
    integrator.u .*= (1-integrator.p[4])                                #--- multiply abundances by 1-intensity
end

perturb_cb = DiscreteCallback(condition_perturb,affect_perturb!)        #--- define the perturbation callback


#--- Definition of the system
times_of_perturbations = (10.,30.,50.,80.)
perturb_intensity = 0.2
p = (r0s, AAs,times_of_perturbations,perturb_intensity) 

#--- Function of derivatives
function ode_LVG_with_pert!(du, u, p, t)
    r0, AA, t_p, p_i = p                     #--- local copy of parameters
    du .= u .* (r0 .+ (AA * u))              #--- equation
end

prob = ODEProblem(ode_LVG_with_pert!, N_init, tspan, p, callback = perturb_cb)        #--- define the problem with callbacks


#--- Run numerical integration   specifying to stop at the times of perturbations
#-----------------------------
sol = solve(prob, tstops = times_of_perturbations, isoutofdomain = (u, p, t) -> any(x -> x < 0, u));

#--- Plot the dynamics
#---------------------
plot(sol, legend=false)

# Inputs and Outputs

In [ ]:
using DelimitedFiles

Random.seed!(1)  

#--- Define the system 
#---------------------
AA = readdlm("./interaction_mat.txt")                #--- get the interaction matrix from a file in working directory
S = size(AA)[1]
r0s = randn(S)
p = (r0s, AA)                                        #--- the parameters
N_init = sample(range(1., 2.,length = 1000),S)       #--- the initial values of state variables
tspan = (0.,100.)                                    #--- min and max values of simulation time
prob = ODEProblem(ode_LVG!, N_init, tspan, p)        #--- define the problem


#--- Run numerical integration
#-----------------------------
sol = solve(prob,isoutofdomain = (u, p, t) -> any(x -> x < 0, u));   



#--- Create a directory for outputs
#----------------------------------
path_out = "./out"
mkpath(path_out)


#--- Save outputs in text files
#------------------------------
writedlm(joinpath(path_out,"abundances.txt"),convert(Array, sol)')
writedlm(joinpath(path_out,"times.txt"),sol.t)


You can also record during simulation by using `open` and `close` functions

In [ ]:
r0_mat = open(joinpath(path_out,"r0mat.txt"),"a")   #--- Open the file

for i in 1:5
    Random.seed!(i)    
    r0s = randn(S) 
    writedlm(r0_mat,r0s')                           #--- Write in the file
end

close(r0_mat)                                       #--- close the file
